# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pickle
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# nltk tools
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Classifiers to try
from sklearn.svm import SVC
from xgboost import XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier

# Additional sklearn tools
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

In [122]:
# load data from database
engine = create_engine('sqlite:///../data/MessagesDatabase.db')
df = pd.read_sql_table('MessagesDatabase', con=engine)

In [123]:
# check that the data read in correctly
df

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26175,The training demonstrated how to enhance micro...,news,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26176,A suitable candidate has been selected and OCH...,news,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26177,"Proshika, operating in Cox's Bazar municipalit...",news,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26178,"Some 2,000 women protesting against the conduc...",news,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
df.shape

(26180, 37)

In [8]:
# View the first message related to water
df[df.water == 1].message.iloc[0]

'Please, we need tents and water. We are in Silo, Thank you!'

In [9]:
# separate the messages from the categories
X = df.message.values
Y = df.iloc[:,2:].values

In [10]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [11]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [12]:
X.shape, Y.shape

((26180,), (26180, 35))

### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    '''
    Convert a string of text into a list of tokens
    
    Input
        text: str, containing the text to be tokenized
        
    Output
        clean_tokens: list, containing normalized, tokenized and lemmatized text
    '''
    
    # detect and replace urls with placeholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    # case normalize and remove punctuation, except for $ sign
    text = re.sub(r'[^a-zA-Z$]', ' ', text.lower())
    
    # tokenize
    tokens = word_tokenize(text)
    
    # remove stop words and lemmatize
    lem = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    clean_tokens = [lem.lemmatize(tok).strip() for tok in tokens if tok not in stop_words]

    return clean_tokens

#### Before we build a model, let's get a sense of how our tokenizer works with sklearn's `TfidfVectorizer`

In [19]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)

In [20]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(X)

Wall time: 17.5 s


In [23]:
# View the first ten elements of the compressed sparse row (csr)
# matrix returned by TfidfVectorizer, showing coordinates and values
print(tfidf_matrix[0])

  (0, 11176)	0.24576657955347114
  (0, 19424)	0.35062973607236964
  (0, 5730)	0.2828895174740424
  (0, 6018)	0.45997200076358874
  (0, 9996)	0.3822004543717484
  (0, 4938)	0.33653829701560334
  (0, 27971)	0.41559934724979647
  (0, 28836)	0.3049570885529014


In [24]:
tfidf_matrix.shape

(26180, 29755)

#### We can see from the shape of `tfidf_matrix` that the lexicon (vocabulary) of the data set after tokenization is 29,755. In other words, the current feature space consists of 29,755 dimensions!

#### We can look at the features that we have for the first message:

In [25]:
# View the features associated with the first message
tfidf_np_matrix = tfidf_matrix.toarray()
nonzero_indices = np.flatnonzero(tfidf_np_matrix[0])
words = tfidf_vectorizer.get_feature_names()
[words[i] for i in nonzero_indices]

['cold', 'could', 'cuba', 'front', 'haiti', 'pas', 'update', 'weather']

In [26]:
# View the first message
df.iloc[0].message

'Weather update - a cold front from Cuba that could pass over Haiti'

#### Our vectorization appears to be working as expected

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(SVC()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
%%time
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=1729)

Wall time: 23.2 ms


In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20944,), (5236,), (20944, 35), (5236, 35))

In [18]:
%%time
pipeline.fit(X_train, y_train)

Wall time: 33min 27s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001C96F8980D0>)),
                ('clf', MultiOutputClassifier(estimator=SVC()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
%%time
y_pred = pipeline.predict(X_test)

Wall time: 2min 30s


In [20]:
print(classification_report(y_test, y_pred, target_names=df.columns.to_list()[2:]))

                        precision    recall  f1-score   support

               related       0.84      0.96      0.89      4002
               request       0.84      0.56      0.67       879
                 offer       0.00      0.00      0.00        26
           aid_related       0.77      0.66      0.71      2177
          medical_help       0.70      0.17      0.27       405
      medical_products       0.87      0.17      0.29       264
     search_and_rescue       0.75      0.09      0.16       137
              security       0.00      0.00      0.00        98
              military       0.86      0.13      0.23       185
                 water       0.74      0.59      0.65       315
                  food       0.81      0.69      0.75       596
               shelter       0.84      0.44      0.58       472
              clothing       0.85      0.45      0.58        74
                 money       0.70      0.06      0.10       127
        missing_people       1.00      

c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [23]:
pickle.dump(pipeline, open('svc_model.pkl', 'wb'))

### 6. Try improving your model. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Now we will try modeling using `XGBRFClassifier`

In [24]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(XGBRFClassifier(use_label_encoder=False, verbosity=0)))
])

In [25]:
%%time
pipeline.fit(X_train, y_train)

Wall time: 3min 51s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001C96F8980D0>)),
                ('clf',
                 MultiOutputClassifier(estimator=XGBRFClassifier(base_score=None,
                                                                 booster=None,
                                                                 colsample_bylevel=None,
                                                                 colsample_bytree=None,
                                                                 enable_categorical=False,
                                                                 gamma=None,
                                                                 gpu_id=None,
                                                                 importance_type=None,
                                                                 interaction_constraints=None,
                                                                 max_delta_step=None,
          

#### This is a significant improvement in the amount of time taken to compute the model. We can see that while it took 33 minutes to learn using support vector classifier, it only took 3 minutes to learn using the cross gradient boosted random forest.

In [26]:
%%time
y_pred = pipeline.predict(X_test)

Wall time: 3.2 s


In [27]:
print(classification_report(y_test, y_pred, target_names=df.columns.to_list()[2:]))

                        precision    recall  f1-score   support

               related       0.79      0.98      0.87      4002
               request       0.82      0.44      0.57       879
                 offer       0.00      0.00      0.00        26
           aid_related       0.72      0.50      0.59      2177
          medical_help       0.60      0.16      0.25       405
      medical_products       0.78      0.29      0.42       264
     search_and_rescue       0.63      0.23      0.33       137
              security       0.33      0.01      0.02        98
              military       0.64      0.15      0.24       185
                 water       0.78      0.55      0.64       315
                  food       0.77      0.78      0.77       596
               shelter       0.80      0.56      0.66       472
              clothing       0.74      0.66      0.70        74
                 money       0.57      0.22      0.32       127
        missing_people       0.58      

c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [28]:
pickle.dump(pipeline, open('xgbrf_model.pkl', 'wb'))

#### Now let's try a random forest without xgboost

In [29]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [30]:
%%time
pipeline.fit(X_train, y_train)

Wall time: 6min 7s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001C96F8980D0>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [31]:
%%time
y_pred = pipeline.predict(X_test)

Wall time: 15.4 s


In [32]:
print(classification_report(y_test, y_pred, target_names=df.columns.to_list()[2:]))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4002
               request       0.84      0.51      0.64       879
                 offer       0.00      0.00      0.00        26
           aid_related       0.75      0.69      0.72      2177
          medical_help       0.74      0.11      0.20       405
      medical_products       0.84      0.10      0.18       264
     search_and_rescue       0.62      0.04      0.07       137
              security       0.00      0.00      0.00        98
              military       0.70      0.09      0.15       185
                 water       0.87      0.41      0.56       315
                  food       0.85      0.54      0.66       596
               shelter       0.84      0.40      0.55       472
              clothing       0.73      0.11      0.19        74
                 money       0.67      0.02      0.03       127
        missing_people       0.00      

c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [33]:
pickle.dump(pipeline, open('rf_model.pkl', 'wb'))

#### Now let's try a neural network

In [43]:
from sklearn.neural_network import MLPClassifier

In [44]:
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(150, 10), random_state=1)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(nn))
])

In [45]:
%%time
pipeline.fit(X_train, y_train)

c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multila

Wall time: 1h 3min 11s


c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001C96F8980D0>)),
                ('clf',
                 MultiOutputClassifier(estimator=MLPClassifier(alpha=1e-05,
                                                               hidden_layer_sizes=(150,
                                                                                   10),
                                                               random_state=1,
                                                               solver='lbfgs')))])

In [46]:
%%time
y_pred = pipeline.predict(X_test)

Wall time: 4.31 s


In [47]:
print(classification_report(y_test, y_pred, target_names=df.columns.to_list()[2:]))

                        precision    recall  f1-score   support

               related       0.85      0.87      0.86      4002
               request       0.58      0.58      0.58       879
                 offer       0.25      0.04      0.07        26
           aid_related       0.68      0.69      0.68      2177
          medical_help       0.39      0.36      0.38       405
      medical_products       0.40      0.28      0.33       264
     search_and_rescue       0.28      0.20      0.23       137
              security       0.08      0.03      0.04        98
              military       0.51      0.36      0.42       185
                 water       0.63      0.65      0.64       315
                  food       0.73      0.67      0.70       596
               shelter       0.61      0.55      0.58       472
              clothing       0.63      0.50      0.56        74
                 money       0.45      0.26      0.33       127
        missing_people       0.22      

c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [48]:
pickle.dump(pipeline, open('nn_model.pkl', 'wb'))

### 6. Improve your model
Use grid search to find better parameters. 

#### Okay, now let's try tuning the xgbrf model since it appears to be the most efficient

In [57]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(XGBRFClassifier(use_label_encoder=False, verbosity=0)))
])

parameters = {
    'tfidf__ngram_range': ((1, 1),(1, 2)),
    'tfidf__max_features': (5000,10000),
    'clf__estimator__learning_rate': (0.01, 0.1),
 }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

In [56]:
XGBRFClassifier().get_xgb_params()

{'colsample_bynode': 0.8,
 'learning_rate': 1.0,
 'reg_lambda': 1e-05,
 'subsample': 0.8,
 'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'interaction_constraints': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'monotone_constraints': None,
 'n_jobs': None,
 'num_parallel_tree': 100,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'scale_pos_weight': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [58]:
%%time
cv.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1), total= 1.8min
[CV] clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1), total= 1.8min
[CV] clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1) 
[CV]  clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1), total= 1.9min
[CV] clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1) 
[CV]  clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1), total= 1.8min
[CV] clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1) 
[CV]  clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 1), total= 1.9min
[CV] clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 2) 
[CV]  clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram_range=(1, 2), total= 1.9min
[CV] clf__estimator__learning_rate=0.01, tfidf__max_features=5000, tfidf__ngram

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 114.4min finished


Wall time: 1h 57min 9s


GridSearchCV(estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x000001C96F8980D0>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=XGBRFClassifier(base_score=None,
                                                                                        booster=None,
                                                                                        colsample_bylevel=None,
                                                                                        colsample_bytree=None,
                                                                                        enable_categorical=False,
                                                                                        gamma=None,
                                                                                        gpu_id=None,
                                                

In [110]:
cv.best_params_

{'clf__estimator__learning_rate': 0.01,
 'tfidf__max_features': 10000,
 'tfidf__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [62]:
%%time
y_pred = cv.predict(X_test)

Wall time: 3.23 s


In [63]:
print(classification_report(y_test, y_pred, target_names=df.columns.to_list()[2:]))

                        precision    recall  f1-score   support

               related       0.79      0.98      0.87      4002
               request       0.82      0.44      0.57       879
                 offer       0.00      0.00      0.00        26
           aid_related       0.72      0.50      0.59      2177
          medical_help       0.59      0.14      0.23       405
      medical_products       0.82      0.30      0.43       264
     search_and_rescue       0.63      0.21      0.32       137
              security       1.00      0.03      0.06        98
              military       0.59      0.17      0.27       185
                 water       0.75      0.55      0.64       315
                  food       0.77      0.78      0.77       596
               shelter       0.80      0.54      0.65       472
              clothing       0.74      0.62      0.68        74
                 money       0.62      0.24      0.34       127
        missing_people       0.62      

c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\embouchero\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [73]:
print('Accuracy of the model: ',(y_test == y_pred).mean())

Accuracy of the model:  0.9452963003383171


#### If we want to see how the classifier does with unseen examples, we provide it a new piece of text to predict labels on:

In [126]:
text = input()
pred = cv.predict([text])
preds = dict(zip(list(df.iloc[:,2:].columns),pred.tolist()[0]))
preds

If you receive this message, please help as soon as you can because a power line went down during the wind storm and many of us need medical care and we are desparately thirsty


{'related': 1,
 'request': 1,
 'offer': 0,
 'aid_related': 1,
 'medical_help': 1,
 'medical_products': 0,
 'search_and_rescue': 0,
 'security': 0,
 'military': 0,
 'water': 1,
 'food': 0,
 'shelter': 0,
 'clothing': 0,
 'money': 0,
 'missing_people': 0,
 'refugees': 0,
 'death': 0,
 'other_aid': 0,
 'infrastructure_related': 0,
 'transport': 0,
 'buildings': 0,
 'electricity': 1,
 'tools': 0,
 'hospitals': 0,
 'shops': 0,
 'aid_centers': 0,
 'other_infrastructure': 0,
 'weather_related': 1,
 'floods': 0,
 'storm': 1,
 'fire': 0,
 'earthquake': 0,
 'cold': 0,
 'other_weather': 0,
 'direct_report': 1}

### 9. Export your model as a pickle file

In [80]:
pickle.dump(cv, open('xgbrf_tuned_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.